# Python-Notebook zum Senden von Lastschrift-Einzugsnachrichten

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')

## Setup

In [ ]:
from datetime import datetime, timedelta

# Get the current year
CURRENT_YEAR = datetime.now().year
print("Current Year:", CURRENT_YEAR)

# Set the update deadline to 7 days from now
UPDATE_DATE = datetime.now() + timedelta(days=7)
print("Update Deadline:", UPDATE_DATE.strftime('%d.%m.%Y'))

# Set the collection date to 14 days from now
COLLECTION_DATE = datetime.now() + timedelta(days=14)
print("Collection Date:", COLLECTION_DATE.strftime('%d.%m.%Y'))

## Import der Mitglieder, Beitröge und Lastschriftmandate

In [ ]:
import pathlib
from src.paths import DATA_DIR
from src.excel import read_excel

# Set the path to the member data file
MEMBER_FILE_PATH = pathlib.Path(DATA_DIR, 'JuBO-Mitglieder.xlsx')
print("File Path:", MEMBER_FILE_PATH)

# Load the members form the "Mitglieder" worksheet
member_data = read_excel(
    workbook_path=MEMBER_FILE_PATH,
    sheet_name="Mitglieder",
    header_map={
        'ID': 'id',
        'Anrede': 'salutation',
        'Vorname': 'first_name',
        'Nachname': 'last_name',
        'E-Mail': 'email',
        'Status': 'status',
        'Mitgliedschaft': 'member_type'
    },
    skip_rows=4
)

# Load the payment information from the "Finanzen" worksheet
payment_data = read_excel(
    workbook_path=MEMBER_FILE_PATH,
    sheet_name="Finanzen",
    header_map={
        'MitgliedsNr.': 'member_id',
        'Beitrag': 'amount_fee',
        'Spende': 'amount_donation',
        'Gesamt': 'amount_total',
        'Voller Name': 'account_holder',
        'Referenz': 'mandate_reference',
        'Gläubiger-ID': 'creditor_id',
        'Erteilt Am': 'issue_date',
        'IBAN (Anonymisiert)': 'iban_anonymized',
        'BIC (Anonymisiert)': 'bic_anonymized',
        'Kreditinstitut': 'credit_institute'
    },
    skip_rows=4
)
# Convert NaN to zero in amount fields and convert the mandate reference to a string
payment_data['amount_donation'] = payment_data['amount_donation'].fillna(0)
payment_data['amount_fee'] = payment_data['amount_fee'].fillna(0)
payment_data['amount_total'] = payment_data['amount_total'].fillna(0)
payment_data['mandate_reference'] = payment_data['mandate_reference'].fillna(0)
payment_data['mandate_reference'] = payment_data['mandate_reference'].astype(int).astype(str)

# Filter members that are still part of the club, i.e. have the status "Aktiv"
active_members = member_data[member_data["status"].isin(["Aktiv"])]

# Merge the active members with the payment data
payments = active_members.merge(
    payment_data,
    how='left',
    left_on='id',
    right_on='member_id'
)
payments

In [ ]:
# Find members that didn't issue a SEPA mandate
payments_with_mandate = payments[payments['issue_date'].notna()]
payments_without_mandate = payments[payments['issue_date'].isna()]
if len(payments_without_mandate) > 0:
    print(f"Found {len(payments_without_mandate)} members without a SEPA mandate. They need to be contacted.")
payments_without_mandate

In [ ]:
# Display the total amounts
print(f'Number of paying members: {len(payments_with_mandate)}/{len(member_data)}')
print(f'Fees:\t\t{payments_with_mandate["amount_fee"].sum()}€')
print(f'Donations:\t{payments_with_mandate["amount_donation"].sum()}€')
print(f'Total:\t\t{payments_with_mandate["amount_total"].sum()}€')

## Vorbereiten der E-Mails

In [ ]:
from src.jinja import read_template

# Set the name of the template file, which is the path relative to the templates directory
TEMPLATE_NAME = 'mitgliedsbeiträge/mitgliedsbeiträge.html.jinja'

# Load the template
template = read_template(TEMPLATE_NAME, globals={
    "format_date": lambda x: x.strftime("%d.%m.%Y"),
    "format_currency": lambda x: "{:,.2f}€".format(x).replace(".", ",")
})

In [ ]:
from src.email import EmailClientConfig, EmailSignatureConfig

# Load the email client and signature configuration from the .env file
client_config = EmailClientConfig.from_dotenv()
signature_config = EmailSignatureConfig.from_dotenv()

In [ ]:
from IPython.display import display, HTML
from src.email import Email

# Prepare the e-mails
emails: list[Email] = []
for payment in payments_with_mandate.itertuples():
    content = template.render(
        salutation=payment.salutation,
        first_name=payment.first_name,
        member_type=payment.member_type,
        amount_fee=payment.amount_fee,
        amount_donation=payment.amount_donation,
        amount_total=payment.amount_total,
        account_holder=payment.account_holder,
        iban_anonymized=payment.iban_anonymized,
        bic_anonymized=payment.bic_anonymized,
        mandate_reference=payment.mandate_reference,
        creditor_id=payment.creditor_id,
        issue_date=payment.issue_date,
        year=CURRENT_YEAR,
        update_date=UPDATE_DATE,
        collection_date=COLLECTION_DATE,
        contact_email=client_config.user,
        signature_name=signature_config.name,
        signature_role=signature_config.role,
        signature_email=signature_config.email,
        signature_phone=signature_config.phone
    )
    email = Email(
        sender=client_config.user,
        to=[payment.email],
            subject=f'JuBO e.V. | Mitgliedsbeitrag { CURRENT_YEAR } | Mitglied Nr. M{ payment.id } { payment.first_name } { payment.last_name }',
        content=content
    )
    emails.append(email)

# Display the first e-mail
email = emails[0]
print("From:", email.sender)
print("To:", email.to)
print("Subject:", email.subject)
display(HTML(email.content))

## Erstellen der E-Mails im Postfach

In [ ]:
# The routine creates the e-mails as drafts, but does not send them - this must
# be done manually by the user. If you want to send the e-mails directly, set
# the SEND_DIRECTLY variable to True.
# Important: This will send the e-mails to the recipients immediately! Only use
# this option if you are absolutely sure that the e-mails are correct and ready
# to be sent.
SEND_DIRECTLY = False

In [ ]:
import time
from tqdm import tqdm
from src.email import EmailClient

# Initialize the email client
email_client = EmailClient(**client_config.__dict__)

# Create the drafts
for email in tqdm(
    emails,
    desc=f"{'Drafting' if not SEND_DIRECTLY else 'Sending'} E-Mails",
    leave=False
):
    email_client.send(email) if SEND_DIRECTLY else email_client.draft(email)
    time.sleep(1)

print(f'Processed {n} e-mails.')